In [58]:
import requests
# import json
import time
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path
# import os

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [59]:

API_BASE = "https://gamma-api.polymarket.com"

def fetch_events(limit=100, offset=0, min_volume=1000000):
    url = (
        f"{API_BASE}/events?"
        f"limit={limit}&offset={offset}&volume_min={min_volume}&active=true&closed=false"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

def fetch_all_events(min_volume=1000000):
    all_events = []
    offset = 0
    limit = 100

    while offset < 600:
        print(f"Fetching offset={offset} ...")

        data = fetch_events(limit=limit, offset=offset, min_volume=min_volume)

        if not data:
            break

        all_events.extend(data)
        offset += limit

        time.sleep(5)

    return all_events


def flatten_events(events):
    event_fields = [
        "id", "slug", "title", "creationDate", "endDate", "startDate", "volume"
    ]

    market_fields = [
        "id", "slug", "question", "startDate", "endDate", "volume", 
        "outcomes", "outcomePrices", "active", "closed", "bestBid", "bestAsk"
    ]

    rows = []

    for event in events:
        event_data = {f"event_{k}": event.get(k) for k in event_fields}

        # --- event-level tags ---
        event_tags = [
            t.get("slug")
            for t in event.get("tags", [])
            if isinstance(t, dict) and t.get("slug")
        ]

        for market in event.get("markets", []):
            market_data = {f"market_{k}": market.get(k) for k in market_fields}

            # --- market-level tags ---
            market_tags = [
                t.get("slug")
                for t in market.get("tags", [])
                if isinstance(t, dict) and t.get("slug")
            ]

            # merge & deduplicate tags
            all_tags = sorted(set(event_tags + market_tags))
            

            row = {
                "market_id": market.get("id"),
                "tags": ",".join(all_tags),
                **event_data,
                **market_data
            }
            rows.append(row)

    
    # Convert to DataFrame
    df = pd.DataFrame(rows)
    df['created_at'] = datetime.now(timezone.utc).replace(microsecond=0).isoformat(timespec='seconds').replace('+00:00', '')

    return df

In [60]:
def main():
    print("Fetching Polymarket events...")
    events = fetch_all_events()
    print(f"Fetched {len(events)} events.")

    # Flatten markets
    df = flatten_events(events)

    old_file = Path("data/polymarket_flat_markets_t0.csv")
    new_file = Path("data/polymarket_flat_markets_t1.csv")

    try:
        # Rename old t1 to t0 (overwrite if exists)
        if new_file.exists():
            new_file.replace(old_file)  # replace() overwrites t0 if it exists
            print(f"Renamed {new_file.name} to {old_file.name}")

        # Save the new CSV as t1
        df.to_csv(new_file, index=False)
        print(f"Saved {len(df)} rows to {new_file.name}")

    except Exception as e:
        print(f"Error occurred: {e}. Saving backup...")
        backup_file = Path("data/polymarket_flat_markets_backup.csv")
        df.to_csv(backup_file, index=False)
        print(f"Saved backup to {backup_file.name}")

In [61]:
main()

Fetching Polymarket events...
Fetching offset=0 ...
Fetching offset=100 ...
Fetching offset=200 ...
Fetched 192 events.
Renamed polymarket_flat_markets_t1.csv to polymarket_flat_markets_t0.csv
Saved 3024 rows to polymarket_flat_markets_t1.csv


Retrieve Fields

In [ ]:

class VolumeAlertService:
    def __init__(self, telegram_token: str, chat_id: str, bid_threshold: float=20, ask_threshold: float=20):
        self.telegram_token = telegram_token
        self.chat_id = chat_id
        self.api_url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
        self.bid_threshold=bid_threshold
        self.ask_threshold=ask_threshold

    def send_message(self, text: str):
        payload = {
            "chat_id": self.chat_id,
            "text": text,
            "parse_mode": "HTML"
        }
        response = requests.post(self.api_url, json=payload)
        response.raise_for_status()
    
    # def build_price_change_message(self, row):
    #     bid_change = row.get("market_bestBid_pct_change", 0)
    #     ask_change = row.get("market_bestAsk_pct_change", 0)
    #     # price_change = row.get("outcome_1_change", 0)
        
    #     messages = []

    #     def format_msg(label, change):
    #         direction = "up" if change > 0 else "down"
    #         arrow = "🟩" if change > 0 else "🔻"
    #         pct = abs(round(change, 2))
    #         return f"{arrow} {label} price went {direction} by {pct}%"

    #     # Bid change
    #     if abs(bid_change) >= self.bid_threshold:
    #         messages.append(format_msg("Bid", bid_change))

    #     # Ask change
    #     if abs(ask_change) >= self.ask_threshold:
    #         messages.append(format_msg("Ask", ask_change))

    #     # If both changed → separate sentences
    #     return "\n".join(messages)

    def build_price_change_message(self, row):
        price_change = row.get("outcome_1_change", 0)
        
        messages = []

        def format_msg(change):
            direction = "up" if change > 0 else "down"
            arrow = "🟩" if change > 0 else "🔻"
            pct = abs(round(change * 100, 2))
            return f"{arrow} Price went {direction} by {pct} percentage points"

        # Bid change
        # if abs(bid_change) >= self.bid_threshold:
        #     messages.append(format_msg("Bid", bid_change))

        # Ask change
        # if abs(ask_change) >= self.ask_threshold:
        #     messages.append(format_msg("Ask", ask_change))

        # If both changed → separate sentences
        messages.append(format_msg(price_change))
        return "\n".join(messages)

    def build_message(self, row):
        event_slug = row.get("event_slug", "")
        link = f"https://polymarket.com/event/{event_slug}"

        #Get Bid/Ask movement message
        price_change_msg = self.build_price_change_message(row)

        # Safely format end date
        end_date = pd.to_datetime(row['market_endDate'], errors='coerce')
        end_date_str = end_date.strftime('%m/%d/%Y') if pd.notna(end_date) else "N/A"

        message = (
            "<b>🔥 Market Update</b>\n\n"
            f"{price_change_msg}\n\n"
            f"<b>Question:</b> {row['market_question']}\n\n"
            f"💹 <b>T1 Best Bid / Ask:</b> {row['market_bestBid']} / {row['market_bestAsk']}\n"
            f"💹 <b>T0 Best Bid / Ask:</b> {row['market_bestBid_t0']} / {row['market_bestAsk_t0']}\n"
            f"📊 <b>Market Volume:</b> {round(row['market_volume']):,}\n"
            f"⏰ <b>Ends:</b> {end_date_str}\n\n"
            f"T0 Price: {row['outcome_1_t0']} | T1 Price: {row['outcome_1']}\n\n"
            # "\n\n"
            f"🔗 <b>Event:</b> {link}\n\n"
            
            # f"T0 Bid Price: {row['market_bestBid_t0']} | T1 Bid Price: {row['market_bestBid']}\n\n"
            # f"T0 Ask Price: {row['market_bestAsk_t0']} | T1 Ask Price: {row['market_bestAsk']}\n\n"
            
        )

        return message

    def process_dataframe(self, df):
        # alert_rows = df[
        #     (df["market_closed"] == False) &
        #     (df["market_active"] == True) &
        #     (df["outcome_1_change"].abs() >= 0.2)
            # (
            #     (df["market_bestBid_pct_change"].abs() >= self.bid_threshold) |
            #     (df["market_bestAsk_pct_change"].abs() >= self.ask_threshold)
            # )
        # ]


        for _, row in df.iterrows():
            msg = self.build_message(row)
            self.send_message(msg)
